# Different data structures

In [105]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

%matplotlib widget

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [106]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dataclasses import dataclass
import jupyter_black

jupyter_black.load()

# Project
from src.data_connectors import read_input_files
from src.data_preparation import search_space

In [107]:
instance = 333
instances_path = "../data/input/HRTInstances"
ins_x = read_input_files.read_file(
    os.path.join(instances_path, f"Instance_{instance}.txt")
)

In [108]:
ins_x.df_setup

,Humans,Robots,WorkingSpaces
0,2,3,3


In [109]:
ins_x.df_resources

,Type,Id
0,H,0
1,H,1
2,R,2
3,R,3
4,R,4


In [110]:
ins_x.df_workingspace_resources

,WorkingSpace,Resource
0,1,2
1,1,0
2,1,1
3,2,3
4,2,0
5,2,1
6,3,4
7,3,0
8,3,1


In [111]:
ins_x.df_workingspace_id.head()

,WorkingSpace,Id
0,1,1
1,1,2
2,1,3
3,1,4
4,1,5


## [Flow](https://miro.com/app/board/uXjVPyl00iw=/)

1. Generate **Initial Population**
2. Verify if order of genes is violating the precedence
    1. Discard those that are violating the precedence
3. Create the array of times (start and end), using the resources time estimator for the respective mode allocation
4. Duplicate it for all the jobs in the same instance (SAC)
5. **Fitness Function**: Compute the total time to complete all the jobs in all working spaces of same instance $\Longrightarrow C$ 
6. **Selection**: Retain the genes with lower C (at maximum X)
    1. Discard those that have larger C
    2. Verify if new population has a significant improvement in total C from previous population.
7. **Generate next population**:
    1. **Crossover**: from survival chromosomes, create offsprings 
    2. **Mutation**: add mutation to the created offsprings (initially larger %, and decreases over time).
8. Return to point **2**.

### 0. The chromosome

```python
chromosome = {
    'mode': [0, 1, 2, 2, 1, ... ], # resource ID
    'order': [2, 3, 1, 4, 5, ...], # order of tasks of each position
    }

```

In [112]:
from src.ga.chromosome import Chromosome

### 1. Generate initial population

By default, it generates population for 1 working space:

**Note**:
- *It is important to generate a large first population, because a lot chromosomes might not survived due to feasibility of solutions.*

In [123]:
from src.ga import first_population

# Modes

possible_modes = first_population.get_possible_modes(ins_x)
# Tasks
n_tasks = first_population.get_total_number_of_tasks_per_working_space(ins_x)

# Chromosomes
chromosomes = first_population.get_first_population(possible_modes, n_tasks)

Possible modes: ['0' '1' '2' '0-2' '1-2']
Number of tasks: 20
Total size of first population: 600


### 2. Verify feasibility of solutions

We need to verify:
- the precedence is valid
- the combination of modes and tasks is possible (the selected mode can perform the associated task) 

In [7]:
df_predecessor_sucessor = ins_x.df_predecessor_sucessor
df_predecessor_sucessor["count"] = 1
df_ps = pd.pivot_table(
    df_predecessor_sucessor,
    index=["Sucessor"],
    columns=["Predecessor"],
    values="count",
)
df_ps["total_dependencies"] = df_ps.sum(axis=0)

In [56]:
df_ps.head(10)

Predecessor,1,2,3,5,6,7,9,11,12,17,...,23,25,26,27,29,31,32,37,38,total_dependencies
Sucessor,,,,,,,,,,,,,,,,,,,,,
2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
3,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
4,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
6,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
7,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
8,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
10,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df_resource_job_time = ins_x.df_resource_job_time
df_res = pd.pivot_table(
    df_resource_job_time, index="Job", columns="Resource", values="Time"
)
df_res["min"] = df_res.min(axis=1)

In [10]:
df_res

Resource,0,1,2,min
Job,,,,
1,13.0,1.0,NaN,1.0
2,35.0,NaN,NaN,35.0
3,5.0,NaN,NaN,5.0
4,30.0,NaN,NaN,30.0
5,99.0,NaN,NaN,99.0
6,36.0,NaN,NaN,36.0
7,41.0,NaN,NaN,41.0
8,40.0,10.0,NaN,10.0
9,20.0,NaN,NaN,20.0
